# 1: Importation des bibliothèques nécessaires

In [235]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report



sns.set_theme(style="whitegrid")
%matplotlib inline

pd.set_option('display.max_columns', None)

# 2: Chargement des données


In [ ]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

print("Aperçu des données d'entraînement :")
print(train_data.head())

print("\nInformations sur les données d'entraînement :")
print(train_data.info())

print("\nValeurs manquantes par colonne :")
print(train_data.isnull().sum())

# 3: Visualisation des données manquantes


In [ ]:
plt.figure(figsize=(10, 6))
sns.heatmap(train_data.isnull(), cbar=False, cmap='viridis')
plt.title("Carte des valeurs manquantes")
plt.show()

# 4: Visualisation initiale des données


In [ ]:
sns.countplot(x='Transported', data=train_data, palette='viridis')
plt.title("Distribution des passagers transportés")
plt.show()

sns.countplot(x='HomePlanet', hue='Transported', data=train_data, palette='viridis')
plt.title("Transported par planète d'origine (HomePlanet)")
plt.show()

sns.countplot(x='Destination', hue='Transported', data=train_data, palette='viridis')
plt.title("Transported par destination")
plt.show()

plt.figure(figsize=(8, 4))
sns.histplot(train_data['Age'], kde=True, bins=30, color='blue')
plt.title("Distribution de l'âge")
plt.show()

train_data['TotalSpending'] = train_data[['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']].sum(axis=1)
plt.figure(figsize=(8, 4))
sns.histplot(train_data['TotalSpending'], kde=True, bins=30, color='green')
plt.title("Distribution des dépenses totales")
plt.show()

sns.countplot(x='CryoSleep', hue='Transported', data=train_data, palette='coolwarm')
plt.title("Transported selon CryoSleep")
plt.show()

train_data['Deck'] = train_data['Cabin'].str.split('/', expand=True)[0]
sns.countplot(x='Deck', hue='Transported', data=train_data, palette='viridis')
plt.title("Transported par deck")
plt.show()

# 5: Data Processing

### Remplir HomePlanet avec la valeur modale

In [239]:
most_frequent_homeplanet = train_data['HomePlanet'].mode()[0]
train_data['HomePlanet'] = train_data['HomePlanet'].fillna(most_frequent_homeplanet)
test_data['HomePlanet'] = test_data['HomePlanet'].fillna(most_frequent_homeplanet)

### CryoSleep : Remplir les valeurs manquantes par False après conversion en booléen

In [240]:
train_data['CryoSleep'] = train_data['CryoSleep'].astype('bool').fillna(False)
test_data['CryoSleep'] = test_data['CryoSleep'].astype('bool').fillna(False)

### Cabin : Extraire Deck et remplir les valeurs manquantes par 'Unknown'

In [241]:
train_data['Deck'] = train_data['Cabin'].str.split('/', expand=True)[0]
test_data['Deck'] = test_data['Cabin'].str.split('/', expand=True)[0]
train_data['Deck'] = train_data['Deck'].fillna('Unknown')
test_data['Deck'] = test_data['Deck'].fillna('Unknown')

### Destination : Remplir les valeurs manquantes par la valeur modale

In [242]:
most_frequent_destination = train_data['Destination'].mode()[0]
train_data['Destination'] = train_data['Destination'].fillna(most_frequent_destination)
test_data['Destination'] = test_data['Destination'].fillna(most_frequent_destination)

### Age : Remplir les valeurs manquantes par la médiane*

In [243]:
median_age = train_data['Age'].median()
train_data['Age'] = train_data['Age'].fillna(median_age)
test_data['Age'] = test_data['Age'].fillna(median_age)

### VIP : Remplir les valeurs manquantes par False après conversion en booléen

In [244]:
train_data['VIP'] = train_data['VIP'].astype('bool').fillna(False)
test_data['VIP'] = test_data['VIP'].astype('bool').fillna(False)

### Dépenses : Remplir les valeurs manquantes par 0 pour RoomService, FoodCourt, ShoppingMall, Spa, VRDeck

In [245]:
expenditures = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
for col in expenditures:
    train_data[col] = train_data[col].fillna(0)
    test_data[col] = test_data[col].fillna(0)

### Supprimer Name car elle n'est pas pertinente

In [246]:
train_data = train_data.drop(columns=['Name'])
test_data = test_data.drop(columns=['Name'])

### Créer TotalSpending (Somme des dépenses)

In [247]:
train_data['TotalSpending'] = train_data[expenditures].sum(axis=1)
test_data['TotalSpending'] = test_data[expenditures].sum(axis=1)

### Calculer GroupSize (taille du groupe à partir de PassengerId)

In [248]:
train_data['GroupSize'] = train_data['PassengerId'].apply(lambda x: int(x.split('_')[0]))
test_data['GroupSize'] = test_data['PassengerId'].apply(lambda x: int(x.split('_')[0]))

# 6:  Finalisation du prétraitement

In [ ]:
print("Aperçu des données prétraitées (entraînement) :")
print(train_data.head())

print("\nAperçu des données prétraitées (test) :")
print(test_data.head())

In [250]:

# Transformation des variables catégoriques en numériques
train_data = pd.get_dummies(train_data, columns=['HomePlanet', 'Destination', 'Deck'], drop_first=True)
test_data = pd.get_dummies(test_data, columns=['HomePlanet', 'Destination', 'Deck'], drop_first=True)

# Alignement des colonnes entre les deux datasets
train_data, test_data = train_data.align(test_data, join='left', axis=1, fill_value=0)

# Définir X (caractéristiques) et y (cible)
features = ['Age', 'CryoSleep', 'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'TotalSpending', 'GroupSize']
X = train_data[features]
y = train_data['Transported'].astype(int)

# Diviser les données en ensembles d'entraînement et de validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# 7: Entraînement du modèle XGBoost

In [ ]:
from xgboost import XGBClassifier

model = XGBClassifier(random_state=42, n_estimators=100, max_depth=10, learning_rate=0.1)
model.fit(X_train, y_train)

# Évaluation du modèle
y_pred = model.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
print(f"Accuracy sur l'ensemble de validation : {accuracy:.2f}")

print("\nRapport de classification :")
print(classification_report(y_val, y_pred))

# Prédictions sur l'ensemble de test
test_data['Transported'] = model.predict(test_data[features]).astype(bool)

# Création du fichier de soumission
submission = test_data[['PassengerId', 'Transported']]
submission.to_csv('submission.csv', index=False)
print("Fichier de soumission 'submission.csv' généré.")

# 8: Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Initialisation du modèle Random Forest
rf_model = RandomForestClassifier(random_state=42, n_estimators=100, max_depth=10)

# Entraîner le modèle sur l'ensemble d'entraînement
rf_model.fit(X_train, y_train)

# Prédictions sur l'ensemble de validation
rf_y_pred = rf_model.predict(X_val)

# Calcul de l'accuracy
rf_accuracy = accuracy_score(y_val, rf_y_pred)
print(f"Accuracy avec Random Forest : {rf_accuracy:.2f}")

# Rapport de classification
print("\nRapport de classification (Random Forest) :")
print(classification_report(y_val, rf_y_pred))

# 9: Gradient Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

# Initialisation du modèle Gradient Boosting
gb_model = GradientBoostingClassifier(random_state=42, n_estimators=100, learning_rate=0.1, max_depth=10)

# Entraînement du modèle
gb_model.fit(X_train, y_train)

# Prédictions sur l'ensemble de validation
gb_y_pred = gb_model.predict(X_val)

# Calcul de l'accuracy
gb_accuracy = accuracy_score(y_val, gb_y_pred)
print(f"Accuracy avec Gradient Boosting : {gb_accuracy:.2f}")

# Rapport de classification
print("\nRapport de classification (Gradient Boosting) :")
print(classification_report(y_val, gb_y_pred))

# 10: Logistic Regression

In [ ]:
# Importation du modèle Logistic Regression
from sklearn.linear_model import LogisticRegression

# Initialisation du modèle Logistic Regression
lr_model = LogisticRegression(random_state=42, max_iter=1000)

# Entraîner le modèle sur l'ensemble d'entraînement
lr_model.fit(X_train, y_train)

# Prédictions sur l'ensemble de validation
lr_y_pred = lr_model.predict(X_val)

# Calcul de l'accuracy
lr_accuracy = accuracy_score(y_val, lr_y_pred)
print(f"Accuracy avec Logistic Regression : {lr_accuracy:.2f}")

# Rapport de classification
print("\nRapport de classification (Logistic Regression) :")
print(classification_report(y_val, lr_y_pred))


# 11: LightGMB

In [ ]:
# Importation de LightGBM
from lightgbm import LGBMClassifier

# Initialisation du modèle LightGBM
lgbm_model = LGBMClassifier(random_state=42, n_estimators=200, learning_rate=0.05, max_depth=-1, num_leaves=31)

# Entraîner le modèle sur l'ensemble d'entraînement
lgbm_model.fit(X_train, y_train)

# Prédictions sur l'ensemble de validation
lgbm_y_pred = lgbm_model.predict(X_val)

# Calcul de l'accuracy
lgbm_accuracy = accuracy_score(y_val, lgbm_y_pred)
print(f"Accuracy avec LightGBM : {lgbm_accuracy:.2f}")

# Rapport de classification
print("\nRapport de classification (LightGBM) :")
print(classification_report(y_val, lgbm_y_pred))


# 12: Stacking Classifier

In [ ]:
# Importation des bibliothèques nécessaires
from sklearn.ensemble import StackingClassifier, RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, classification_report

# Définir les modèles de base
base_models = [
    ('random_forest', RandomForestClassifier(random_state=42, n_estimators=100, max_depth=10)),
    ('gradient_boosting', GradientBoostingClassifier(random_state=42, n_estimators=100, max_depth=10, learning_rate=0.1)),
    ('lightgbm', LGBMClassifier(random_state=42, n_estimators=100, learning_rate=0.1, max_depth=-1, num_leaves=31))
]

# Définir le méta-modèle
meta_model = LogisticRegression(random_state=42, max_iter=1000)

# Initialisation du Stacking Classifier
stacking_model = StackingClassifier(estimators=base_models, final_estimator=meta_model, cv=5)

# Entraîner le modèle sur l'ensemble d'entraînement
stacking_model.fit(X_train, y_train)

# Prédictions sur l'ensemble de validation
stacking_y_pred = stacking_model.predict(X_val)

# Calcul de l'accuracy
stacking_accuracy = accuracy_score(y_val, stacking_y_pred)
print(f"Accuracy avec Stacking Classifier (avec LightGBM) : {stacking_accuracy:.2f}")

# Rapport de classification
print("\nRapport de classification (Stacking Classifier avec LightGBM) :")
print(classification_report(y_val, stacking_y_pred))


### La méthode LightGBM est la plus intéressaante dans notre cas d'usage, elle retourne les meilleurs résultats ainsi qu'un temps très compétitif.`